## prompt test notebook

In [1]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain_huggingface import HuggingFacePipeline
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_experimental.text_splitter import SemanticChunker
from pdf_preprocessing import LegalText

# 캐시 디렉토리 설정
cache_dir = './weights'
os.makedirs(cache_dir, exist_ok=True)
os.environ['TRANSFORMERS_CACHE'] = cache_dir
os.environ['HF_HOME'] = cache_dir
os.environ['HF_DATASETS_CACHE'] = os.path.join(cache_dir, 'datasets')
os.environ['HUGGINGFACE_HUB_CACHE'] = cache_dir
os.environ['TORCH_HOME'] = os.path.join(cache_dir, 'torch')


In [1]:
from langchain_core.prompts import PromptTemplate

template = "{country}의 수도는 어디인가요?"
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [4]:
from langchain import hub

# 가장 최신 버전의 프롬프트를 가져옵니다.
prompt = hub.pull("rlm/rag-prompt")
print(prompt)

/Users/minsuson/.pyenv/versions/3.10.9/envs/service-env/lib/python3.10/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


input_variables=['context', 'question'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


In [2]:
if torch.backends.mps.is_available():
    device = 'mps'
elif torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

def load_model(model_name):
    if model_name == 'llama':
        tokenizer = AutoTokenizer.from_pretrained(
            "davidkim205/Ko-Llama-3-8B-Instruct",
            cache_dir=cache_dir
            )
        model = AutoModelForCausalLM.from_pretrained(
            "davidkim205/Ko-Llama-3-8B-Instruct",
            device_map="auto",
            torch_dtype=torch.float16,
            cache_dir=cache_dir
            )
    elif model_name == 'qwen': 
        tokenizer = AutoTokenizer.from_pretrained(
            "davidkim205/Ko-Qwen-3-8B-Instruct",
            cache_dir=cache_dir
            )
        model = AutoModelForCausalLM.from_pretrained(
            "davidkim205/Ko-Qwen-3-8B-Instruct",
            device_map="cuda",
            torch_dtype=torch.float16,
            cache_dir=cache_dir
            )
        
    return tokenizer, model

def load_embedding(model_name, device):
    if model_name == 'bge':
        model_name = "upskyy/bge-m3-korean"
        embeddings = HuggingFaceEmbeddings(
            model_name=model_name,
            model_kwargs={'device': device},
            encode_kwargs={'normalize_embeddings': True},
            cache_folder=cache_dir
        )
        return embeddings
    else:
        assert False, f"Unknown model name: {model_name}"


def load_doc(runpod):
    if runpod:
        pdf_path = "/workspace/LangEyE/crawling/장애인복지법.pdf"
        docs = LegalText(pdf_path).documents
    else:
        pdf_path = "/Volumes/MINDB/24년/SW아카데미/LangEyE/crawling/장애인복지법.pdf"
        docs = LegalText(pdf_path).documents
        
    return docs

In [3]:
embeddings = load_embedding('bge', device)
tokenizer, model = load_model('llama')
docs = load_doc(True)

text_splitter = SemanticChunker(embeddings)
splits = text_splitter.split_documents(docs)

/tmp/ipykernel_634/2991526531.py:37: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
embeddings = load_embedding('bge', device)
tokenizer, model = load_model('llama')
docs = load_doc(True)


/tmp/ipykernel_6011/2991526531.py:37: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [15]:
temp_rag = docs[85:86]
temp_rag

[Document(metadata={'law_title': '', 'effective_date': '2024-10-22', 'paragraph': '제5장 복지시설과 단체', 'article_number': '제59조의9(금지행위)', 'document_type': '법률', 'is_valid': True, 'legal_area': '장애인복지'}, page_content='누구든지 다음 각 호의 어느 하나에 해당하는 행위를 하여서는 아니 된다. <개정 2017. 2. 8.>\n1. 장애인에게 성적 수치심을 주는 성희롱ㆍ성폭력 등의 행위\n2. 장애인의 신체에 폭행을 가하거나 상해를 입히는 행위\n2의2. 장애인을 폭행, 협박, 감금, 그 밖에 정신상 또는 신체상의 자유를 부당하게 구속하는 수단으로써 장애인의 자\n유의사에 어긋나는 노동을 강요하는 행위\n3. 자신의 보호ㆍ감독을 받는 장애인을 유기하거나 의식주를 포함한 기본적 보호 및 치료를 소홀히 하는 방임행위\n4. 장애인에게 구걸을 하게 하거나 장애인을 이용하여 구걸하는 행위\n5. 장애인을 체포 또는 감금하는 행위\n6. 장애인의 정신건강 및 발달에 해를 끼치는 정서적 학대행위\n7. 장애인을 위하여 증여 또는 급여된 금품을 그 목적 외의 용도에 사용하는 행위\n8. 공중의 오락 또는 흥행을 목적으로 장애인의 건강 또는 안전에 유해한 곡예를 시키는 행위\n[전문개정 2015. 6. 22.]\n[제59조의7에서 이동, 종전 제59조의9는 제59조의11로 이동 <2017. 12. 19.>]\n제59조의10(장애인학대의 예방과 방지 의무) 국가와 지방자치단체는 장애인학대의 예방과 방지를 위하여 다음 각 호의\n조치를 취하여야 한다.\n1. 장애인학대의 예방과 방지를 위한 각종 정책의 수립 및 시행\n2. 장애인학대의 예방과 방지를 위한 연구ㆍ교육ㆍ홍보와 장애인학대 현황 조사\n3. 장애인학대에 관한 신고체계의 구축ㆍ운영\n4. 장애인학대로 인하여 피해를 입은 장애인(이하 “피해장애인”이라 한다)의 보호 및 

In [16]:
vectorstore = FAISS.from_documents(
    documents=temp_rag,
    embedding=embeddings
)

retriever = vectorstore.as_retriever(
    search_kwargs={
        "k": 3,  # 가장 관련성 높은 3개 결과
        "score_threshold": 0.5  # 유사도 0.5 이상만 반환
    }
)


pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.7,
        top_p=0.95,
        repetition_penalty=1.15
    )
llm = HuggingFacePipeline(pipeline=pipe)


Device set to use cuda:0


In [20]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate


def format_docs(docs):
    if not docs:
        return {
            "context": "",
            "law_title": "",
            "document_type": "",
            "effective_date": "",
            "legal_area": ""
        }
        
    formatted_texts = []
    first_meta = docs[0].metadata
    
    for doc in docs:
        metadata = doc.metadata
        formatted_text = f"""
            [법령 위치]
            법률명: {metadata['law_title']}
            장: {metadata['paragraph'].strip()}
            조문: {metadata['article_number'].strip()}
            
            [조문 내용]
            {doc.page_content.strip()}
        """
        formatted_texts.append(formatted_text)
    
    return {
        "context": "\n\n---\n\n".join(formatted_texts),
        "law_title": first_meta["law_title"],
        "document_type": first_meta["document_type"],
        "effective_date": first_meta["effective_date"],
        "legal_area": first_meta["legal_area"]
    }

template = """장애인복지법 관련 질의응답을 진행하겠습니다.
[법령 기본 정보]
법률명: {law_title}
문서 유형: {document_type}
시행일자: {effective_date}
법률 분야: {legal_area}

[참고할 법령 내용]
{context}

[질문]
{question}

[답변 규칙]
1. 위 법령 내용만을 기반으로 답변합니다.
2. 답변의 근거가 되는 조항(예: 제00조 제0항)을 반드시 먼저 명시합니다.
3. 조항의 내용을 직접 인용하면서 설명합니다.
4. 법령에 명시되지 않은 내용은 추정하거나 해석하지 않고, "해당 내용은 제시된 법령에 명시되어 있지 않습니다"라고 답변합니다.

[답변]"""

prompt = PromptTemplate.from_template(template)

def combine_metadata(retrieved_docs, original_query):
    doc_metadata = retrieved_docs
    return {
        "law_title": doc_metadata["law_title"],
        "document_type": doc_metadata["document_type"],
        "effective_date": doc_metadata["effective_date"],
        "legal_area": doc_metadata["legal_area"],
        "context": doc_metadata["context"],
        "question": original_query
    }

# 체인 구성 수정
retrieval_chain = {
    "original_query": RunnablePassthrough(),
    "retrieved_docs": retriever | RunnableLambda(format_docs)
}

# RAG 체인 생성 (사용자 정보 포함)
rag_chain = (
    {
        "context": get_context,
        "question": lambda x: x if isinstance(x, str) else x["question"]
    }
    | prompt
    | llm
    | StrOutputParser()
)

test_question = "장애인복지법 제59조의9에는 무엇이 명시되어 있나요?"
answer = rag_chain.invoke({"question": test_question})
print(test_question)
print(answer)


장애인복지법 관련 질의응답을 진행하겠습니다.
[법령 기본 정보]
법률명: 
문서 유형: 
시행일자: 
법률 분야: 

[참고할 법령 내용]


[질문]
금지행위에는 무엇이 있나요?

[답변 규칙]
1. 위 법령 내용만을 기반으로 답변합니다.
2. 답변의 근거가 되는 조항(예: 제00조 제0항)을 반드시 먼저 명시합니다.
3. 조항의 내용을 직접 인용하면서 설명합니다.
4. 법령에 명시되지 않은 내용은 추정하거나 해석하지 않고, "해당 내용은 제시된 법령에 명시되어 있지 않습니다"라고 답변합니다.

[답변] 
금지행위는 장애인복지법에서 다음과 같이 정의되고 있습니다.
제1조 (목적 및 범위) 이 법은 장애인의 복지를 증진하고 장애인을 위한 시설, 서비스 등을 보장함으로써 사회통합을 실현하는 것을 목적으로 한다. 이 법은 장애인이 직면하는 차별과 혐오로부터 보호받을 수 있도록 하며, 장애인들을 존중하며 그 권리와 의무를 인정한다.

따라서, 장애인복지법에서는 직접적인 금지행위를 명시하고 있지는 않지만, 장애인들의 권리를 보호하고 사회통합을 위해 다양한 정책과 시설을 제공하는데 초점을 맞추고 있습니다. 만약 특정 상황에서 금지되는 행위나 행동이 있다면 해당 상황에 따라 다른 법령이나 규정을 참고해야 합니다.


In [11]:
# 출력 포맷팅을 위한 함수들
import re
from typing import Dict, List

def format_legal_response(response: str) -> Dict:
    """
    법률 답변을 구조화된 형태로 파싱하는 함수
    
    Returns:
        {
            'references': [참조 조항들],
            'summary': 요약 답변,
            'details': 상세 설명,
            'limitations': 한계점이나 주의사항
        }
    """
    try:
        # 참조 조항 추출 (예: 제00조 제0항, 제0호 등)
        references = re.findall(r'제\d+조(?:\s*제\d+항)?(?:\s*제\d+호)?', response)
        
        # 답변 본문에서 참조 조항 제외한 실제 설명 부분 추출
        details = response.split('\n')
        main_content = [line for line in details if not line.startswith('제')]
        
        # 한계점이나 주의사항 확인 (법령에 명시되지 않은 내용 등)
        limitations = []
        if "명시되어 있지 않습니다" in response:
            limitations.append("일부 내용은 제시된 법령에서 확인할 수 없습니다.")
            
        return {
            'references': list(set(references)),  # 중복 제거
            'summary': main_content[0] if main_content else "",
            'details': '\n'.join(main_content[1:]) if len(main_content) > 1 else "",
            'limitations': limitations
        }
    except Exception as e:
        return {
            'error': f"응답 파싱 중 오류 발생: {str(e)}",
            'raw_response': response
        }

def print_formatted_response(parsed_response: Dict):
    """
    파싱된 응답을 보기 좋게 출력하는 함수
    """
    print("=" * 80)
    print("📚 참조 조항:")
    if parsed_response.get('references'):
        for ref in parsed_response['references']:
            print(f"  • {ref}")
    else:
        print("  • 명시적 참조 조항 없음")
    
    print("\n📌 답변 요약:")
    print(f"  {parsed_response.get('summary', '요약 없음')}")
    
    if parsed_response.get('details'):
        print("\n📝 상세 설명:")
        print(parsed_response['details'])
    
    if parsed_response.get('limitations'):
        print("\n⚠️ 주의사항:")
        for limitation in parsed_response['limitations']:
            print(f"  • {limitation}")
    print("=" * 80)

# RAG 체인 수정
def process_rag_response(question: str, response: str):
    """
    RAG 체인의 응답을 처리하고 포맷팅하는 함수
    """
    parsed = format_legal_response(response)
    print(f"❓ 질문: {question}\n")
    print_formatted_response(parsed)
    return parsed

# 사용 예시:
test_question = "장애인복지법 제59조의9에는 무엇이 명시되어 있나요?"
answer = rag_chain.invoke({"question": test_question})
process_rag_response(test_question, answer)

❓ 질문: 장애인복지법 제59조의9에는 무엇이 명시되어 있나요?

📚 참조 조항:
  • 제11조
  • 제00조 제0항
  • 제59조

📌 답변 요약:
  장애인복지법 관련 질의응답을 진행하겠습니다.

📝 상세 설명:

참고할 법령 내용:
9.>
1. 장애인복지정책의 기본방향에 관한 사항
2. 장애인복지 향상을 위한 제도개선과 예산지원에 관한 사항
3. 중요한 특수교육정책의 조정에 관한 사항
4. 장애인 고용촉진정책의 중요한 조정에 관한 사항
5. 장애인 이동보장 정책조정에 관한 사항
6. 장애인정책 추진과 관련한 재원조달에 관한 사항
7. 장애인복지에 관한 관련 부처의 협조에 관한 사항
7의2. 다른 법령에서 위원회의 심의를 거치도록 한 사항
8. 그 밖에 장애인복지와 관련하여 대통령령으로 정하는 사항
③위원회는 필요하다고 인정되면 관계 행정기관에 그 직원의 출석ㆍ설명과 자료 제출을 요구할 수 있다. ④위원회는 제2항의 사항을 미리 검토하고 관계 기관 사이의 협조 사항을 정리하기 위하여 위원회에 장애인정책조
정실무위원회(이하 “실무위원회”라 한다)를 둔다. ⑤위원회와 실무위원회의 구성ㆍ운영에 관하여 필요한 사항은 대통령령으로 정한다.

30., 2019. 1. 15.>
⑥제2항에 따른 장애인복지시설의 시설기준ㆍ신고ㆍ변경신고 및 이용 등에 관하여 필요한 사항은 보건복지부령으
로 정한다.<개정 2008. 2. 29., 2010. 1. 18., 2011. 3. 30., 2019. 1. 15.>
 

① 제59조의4에 따라 장애인학대 신고를 접수한 장애인권익옹호기관의 직원이나 사법경
찰관리는 지체 없이 장애인학대현장에 출동하여야 한다. 이 경우 장애인권익옹호기관의 장이나 수사기관의 장은 서
로 동행하여 줄 것을 요청할 수 있으며, 그 요청을 받은 장애인권익옹호기관의 장이나 수사기관의 장은 정당한 사유
가 없으면 소속 직원이나 사법경찰관리가 현장에 동행하도록 하여야 한다. <개정 2015.

① 누구든지 장애인학대 및 장애인 대상 성범죄를 알
게 된 때에는 제59

{'references': ['제11조', '제00조 제0항', '제59조'],
 'summary': '장애인복지법 관련 질의응답을 진행하겠습니다.',
 'details': '\n참고할 법령 내용:\n9.>\n1. 장애인복지정책의 기본방향에 관한 사항\n2. 장애인복지 향상을 위한 제도개선과 예산지원에 관한 사항\n3. 중요한 특수교육정책의 조정에 관한 사항\n4. 장애인 고용촉진정책의 중요한 조정에 관한 사항\n5. 장애인 이동보장 정책조정에 관한 사항\n6. 장애인정책 추진과 관련한 재원조달에 관한 사항\n7. 장애인복지에 관한 관련 부처의 협조에 관한 사항\n7의2. 다른 법령에서 위원회의 심의를 거치도록 한 사항\n8. 그 밖에 장애인복지와 관련하여 대통령령으로 정하는 사항\n③위원회는 필요하다고 인정되면 관계 행정기관에 그 직원의 출석ㆍ설명과 자료 제출을 요구할 수 있다. ④위원회는 제2항의 사항을 미리 검토하고 관계 기관 사이의 협조 사항을 정리하기 위하여 위원회에 장애인정책조\n정실무위원회(이하 “실무위원회”라 한다)를 둔다. ⑤위원회와 실무위원회의 구성ㆍ운영에 관하여 필요한 사항은 대통령령으로 정한다.\n\n30., 2019. 1. 15.>\n⑥제2항에 따른 장애인복지시설의 시설기준ㆍ신고ㆍ변경신고 및 이용 등에 관하여 필요한 사항은 보건복지부령으\n로 정한다.<개정 2008. 2. 29., 2010. 1. 18., 2011. 3. 30., 2019. 1. 15.>\n \n\n① 제59조의4에 따라 장애인학대 신고를 접수한 장애인권익옹호기관의 직원이나 사법경\n찰관리는 지체 없이 장애인학대현장에 출동하여야 한다. 이 경우 장애인권익옹호기관의 장이나 수사기관의 장은 서\n로 동행하여 줄 것을 요청할 수 있으며, 그 요청을 받은 장애인권익옹호기관의 장이나 수사기관의 장은 정당한 사유\n가 없으면 소속 직원이나 사법경찰관리가 현장에 동행하도록 하여야 한다. <개정 2015.\n\n① 누구든지 장애인학대 및 장애인 대상 성범죄를 알\n게 된 때에는 제59